# <center><font color=slate>Closures and Decorators</font></center>

## <center><font color=tomato>Returning Functions from Functions</font></center>


### <center><font color=lightGreen>First class functions:</font></center>
Functions can be treated like any other object


```
def outer():
    def inner():
        print("inner")
    return inner

i = outer()
i() -> "inner" ```

example:

In [700]:
def enclosing():
    def local_func():
        print("local_func")
    return local_func

lf = enclosing()
lf()

local_func


## <center><font color=tomato>Closures</font></center>
Maintain references to objects from earlier scopes


In [701]:
def enclosing():
    x = 'closed over'
    def local_func():
            print(x)
    return local_func
lf = enclosing()
lf()

closed over


In [702]:
lf.__closure__

(<cell at 0x7fed5d193b20: str object at 0x7fed5c6cfe30>,)

## <center><font color=tomato>Function Factory</font></center>
Functions that returns newm specialized functions


In [703]:
def raise_to(exp):
    def raise_to_exp(x):
        return pow(x,exp)
    return raise_to_exp

square = raise_to(2)
square.__closure__

(<cell at 0x7fed5c61b7f0: int object at 0x107dc0ac0>,)

In [704]:
square(9)

81

In [705]:
square(2)

4

In [706]:
cube = raise_to(3)
cube(3)

27

In [707]:
cube(2)

8

## </center><font color=tomato>Nonlogical Keywords</font></center>


In [708]:
message = 'global'

def enclosing():
    message = 'enclosing'

    def local():
        message = 'local'

    print('enclosing message', message)
    local()
    print('enclosing message', message)

print('global message', message)
enclosing()
print('global message', message)


global message global
enclosing message enclosing
enclosing message enclosing
global message global



## <center>`global`</center>
Introduces names from `global` namespace into the `local` namespace


In [709]:
message = 'global'


def enclosing():
    message = 'enclosing'

    def local():
        global message
        message = 'local'

    print('enclosing message', message)
    local()
    print('enclosing message', message)


print('global message', message)
enclosing()
print('global message', message)



global message global
enclosing message enclosing
enclosing message enclosing
global message local


## <center>`nonlocal`</center>
Introduces names from the enclosing namespace into the local namespace


In [710]:
message = 'global'

def enclosing():
    message = 'enclosing'

    def local():
        nonlocal message
        message = 'local'

    print('enclosing message', message)
    local()
    print('enclosing message', message)


print('global message', message)
enclosing()
print('global message', message)


global message global
enclosing message enclosing
enclosing message local
global message global


timer example

In [711]:
import time


def make_timer():
    last_called = None

    def elapsed():
        nonlocal last_called
        now = time.time()
        if last_called is None:
            last_called = now
            return None
        result = now - last_called
        last_called = now
        return result

    return elapsed

In [712]:
t = make_timer()

In [713]:
t()

In [714]:
t()

0.004183053970336914

In [715]:
t()

0.0051708221435546875

In [716]:
t()

0.0047681331634521484

## <center><font color=tomato>Decorators - functions</font></center>

-   Modify or enhance functions without changing their definition
-   Implemented as callables that take and return other callables
-   Replace, enhance, or modify existing functions
-   Does not change the original function definition
-   Calling code does not need to change
-   Decorator mechanism uses the modified function's original name

<h2>

```
@my_decorator
def my_function():
    ...```


In [717]:
from escapeUnicode import fruit
fruit()

"'Pi\\xf1a'"

## <center><font color=tomato>Classes as decorators</font></center>

<h2>

```
class MyDec:
    def __init__(self, f):
        ...
    def __call__(self):
        ...

@MyDec
def func()
    ...
```
</h2>

to better explain this we will use the file `CallCount.py`

In [718]:
class CallCount:
    def __init__(self, f):
        self._f = f
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        return self._f(*args, **kwargs)


@CallCount
def hello(name):
    print('Hello, {}'.format(name))

hello('Cami')

Hello, Cami


In [719]:
hello('Nico')

Hello, Nico


In [720]:
hello('Mao')

Hello, Mao


In [721]:
hello.count


3

## <center><font color=tomato>Instances as Decorators</font></center>

```
class anotherDec:
    def __call__(self, f):
        def wrap:
            ...
        return wrap

@AnotherDec()
def func():
    ...
```

to better explain this we will use the file `tracer.py`

In [722]:
class Tracer:
    def __init__(self):
        self.enabled = True

    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self.enabled:
                print('Calling {}'.format(f))
            return f(*args, **kwargs)
        return wrap


tracer = Tracer()

@tracer
def rotate_list(l):
    return l[1:] + [l[0]]

In [723]:
l = [1, 2, 3]
l = rotate_list(l)

Calling <function rotate_list at 0x7fed5c9a8550>


In [724]:
l

[2, 3, 1]

In [725]:
l = rotate_list(l)

Calling <function rotate_list at 0x7fed5c9a8550>


In [726]:
l

[3, 1, 2]

In [727]:
type(l)

list

In [728]:
tracer.enabled = False
l = rotate_list(l)

In [729]:
l


[1, 2, 3]

## <center><font color=tomato>Multiple decorators</font></center>

```
@decorator1
@decorator2
@decorator3
def some_function():
    ...
```
-   It is possible to use multiple decorators in a single function
-   Decorators will be executed in reverse order, in the example above `decorator3`
will be applied to `some_function` first, then the result it will be applied
 `@decorator2` and then `@decorator1`


For the next example we will use 2 previous decorators `escape_unicode()` and `tracer()`
to better explain this we will use the file `islandMaker.py`

In [730]:
def escapeUnicode(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return ascii(x)

    return wrap


class Tracer:
    def __init__(self):
        self.enabled = True

    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self.enabled:
                print('Calling {}'.format(f))
            return f(*args, **kwargs)

        return wrap


tracer = Tracer()


@tracer
@escapeUnicode
def norwegianIslandMaker(name):
    return name + 'øy'


In [731]:
norwegianIslandMaker('Llama')

Calling <function escapeUnicode.<locals>.wrap at 0x7fed5c9a8820>


"'Llama\\xf8y'"

In [732]:
tracer.enabled = False
norwegianIslandMaker('Viva')


"'Viva\\xf8y'"

## <center><font color=tomato>Decorating methods</font></center>


In [733]:

class IslandMaker:
    def __init__(self, suffix):
        self._suffix = suffix

    @tracer
    def make_island(self, name):
        return name + self._suffix

tracer.enabled = True
im = IslandMaker(' Island')
im.make_island('Python')

Calling <function IslandMaker.make_island at 0x7fed5d0ee9d0>


'Python Island'

In [734]:
im.make_island('Llama')

Calling <function IslandMaker.make_island at 0x7fed5d0ee9d0>


'Llama Island'

In [735]:
tracer.enabled = False
im.make_island('Viva')

'Viva Island'

## <center><font color=tomato>Keep metadata from original functions</font>

`functools.wrap()`</h2>

To keep metadata information (docstrings and name) from the original function
this method needs to be called as follows:

In [736]:

import functools

def escapeUnicode(f):
    def wrap(*args, **kwargs):
        """This is the function wraps"""
        x = f(*args, **kwargs)
        return ascii(x)
    return wrap

@escapeUnicode
def fruit():
    """This is the function fruit"""
    return 'Piña'

help(fruit)

Help on function wrap in module __main__:

wrap(*args, **kwargs)
    This is the function wraps



In [737]:
fruit.__name__


'wrap'

In [738]:
def escapeUnicode(f):
    @functools.wraps(f)
    def wrap(*args, **kwargs):
        """This is the function wraps"""
        x = f(*args, **kwargs)
        return ascii(x)
    return wrap

@escapeUnicode
def vegetable():
    """This is the function vegetable"""
    return 'Ñame'
help(vegetable)

Help on function vegetable in module __main__:

vegetable()
    This is the function vegetable



In [739]:
vegetable.__name__

'vegetable'

One use of decorators is to validate arguments in a function,
the following example doesn't look as a standard decorator because
decorator function has a argument which is not a function but an
integer, Python recognizes this way, this integer refers to the index number that has to be validated:

In [740]:
def checkNonNegative(index):
    def validator(f):
        def wrap(*args):
            if args[index] < 0:
                raise ValueError('Argument {} must be non-negative'.format(index))
            return f(*args)
        return wrap
    return validator

@checkNonNegative(1)
def createListValue(value, size):
    return [value] * size

createListValue(-2,4)

[-2, -2, -2, -2]

In [741]:
try:
    createListValue(2,-1)
except ValueError as e:
    print(e)


Argument 1 must be non-negative
